# Test openvisuspy dashboards


In [ ]:
import os,sys
import panel as pn

print(sys.executable)
print(pn.__version__)
pn.extension("ipywidgets", "floatpanel","codeeditor",
             log_level="DEBUG",
             notifications=True, 
             sizing_mode="stretch_width"
)

If you have problems running the following cell,  see # https://github.com/holoviz/holoviews/issues/4861
  - Restart and Clear All Cells 
  - save the notebook 
  - kill jupyter lab 
  - restart

In [ ]:
button = pn.widgets.Button(name="Is Panel working? Click me...")
def onClick(evt):
    button.name="Yes"
button.on_click(onClick)
app=pn.Row(button)
app.servable()

Import and configure OpenVisus

In [ ]:
endpoint_url="https://maritime.sealstorage.io/api/v0/s3"

datasets={
     "datasets": [
         {"name":"david",  "url":"http://atlantis.sci.utah.edu/mod_visus?dataset=david_subsampled&cached=idx"},
         {"name":"2kbit1", "url":"http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1&cached=idx"},
         {"name":"retina", "url":"http://atlantis.sci.utah.edu/mod_visus?dataset=rabbit&cached=idx"},
         {
             "name":"chess-zip","url":"http://atlantis.sci.utah.edu:80/mod_visus?dataset=chess-zip&cached=idx",
             "palette" :"Viridis256", "range-min": -0.017141795, "range-max": +0.012004322,
         },
         {
             "name":"chess-recon","url":"http://atlantis.sci.utah.edu:80/mod_visus?dataset=chess-recon_combined_1_2_3_fullres_zip&cached=idx",
             "palette" :"Plasma256", "range-min": -0.0014, "range-max": +0.0020,   
         },
         {
            "name": "llc2160_arco","url": f"{endpoint_url}/utah/nasa/dyamond/mit_output/llc2160_arco/visus.idx?cached=idx& access_key=any&secret_key=any&endpoint_url={endpoint_url}",
            "palette":"colorcet.coolwarm", "range-min":-0.25256651639938354, "range-max":+0.3600933849811554,
            "timestep-delta":10, "timestep": 2015, "resolution": -6, 
         },
         {
            "name":"bellows", "url": "http://atlantis.sci.utah.edu/mod_visus?dataset=bellows_CT_NASA_JHochhalter&cached=idx",
            "palette":"Greys256", "range-min":0, "range-max":65536
         } 
         ] + [ 
            {
                "name": f"diamond-{zone}",   "url": f"{endpoint_url}/utah/nasa/dyamond/idx_arco/face{zone}/u_face_{zone}_depth_52_time_0_10269.idx?cached=idx& access_key=any&secret_key=any&endpoint_url={endpoint_url}",
                "palette": "Turbo256", "range-min":-30.0, "range-max":60.0,
                "timestep-delta":10, "resolution": -6,  "directions": {'Long':0, 'Lat':1, 'Depth':2},
                "logic-to-physic":[(0.0,1.0), (0.0,1.0), (0.0,10.0)],               
            }
            for zone in range(6)
        ] 
    }

print("datasets defined")

In [ ]:
import os,sys,logging,time
import numpy as np

os.environ["BOKEH_ALLOW_WS_ORIGIN"]="*"
os.environ["BOKEH_LOG_LEVEL"]="debug"
os.environ["VISUS_CPP_VERBOSE"]="0"
os.environ["VISUS_NETSERVICE_VERBOSE"]="0"
os.environ["VISUS_VERBOSE_DISKACCESS"]="0"

sys.path.append("c:/projects/openvisus/build/RelWithDebInfo")
sys.path.append("c:/projects/openvisuspy/src")

from openvisuspy import Slice, SetupJupyterLogger, LoadDataset, ExecuteBoxQuery
logger=SetupJupyterLogger(logging_level=logging.DEBUG) 
print(time.time(),"OpenVisus imported")

# Show single slice of a RGB 2D dataset (David)

In [ ]:
def CreateApp():
    view=Slice()
    view.load(datasets)
    return pn.Column(view.getMainLayout(),sizing_mode="stretch_width",height=800)

app=CreateApp()
app.servable()

# Example: OpenVisus API to read data

In [ ]:
llc2160_url=[it for it in datasets["datasets"] if it["name"]=="llc2160_arco"][0]["url"]
db=LoadDataset(llc2160_url)
W,H,D=db.getLogicBox()[1]
access=db.createAccess()

# get a Z slice in the middle to compute the range of the data
endh=db.getMaxResolution()-6
Z=D//2
logic_box, delta, num_pixels=db.getAlignedBox([[0,0,Z],[W,H,Z]], endh, slice_dir=2)
data=list(ExecuteBoxQuery(db, access=access, logic_box=logic_box,  endh=endh, num_refinements=1))[0]['data']
palette_range = np.min(data)/4, np.max(data)/4  
print("logic box",W,H,D) 
print("palette_range",palette_range)
print("resolution",db.getMaxResolution())

In [ ]:
menu_items = [('Option A', 'a'), ('Option B', 'b'), ('Option C', 'c'), None, ('Help', 'help')]

menu_button = pn.widgets.MenuButton(name='Dropdown', items=menu_items, button_type='primary')

pn.Column(menu_button, height=200)